In [1]:
import numpy as np
import keras
from numpy import genfromtxt
from keras.models import Sequential
from keras.layers import Dense, Dropout, Input
from keras.models import Model

train = genfromtxt('train_solved.csv', delimiter=',')
test = genfromtxt('test_solved.csv', delimiter=',')


Using TensorFlow backend.


In [2]:
# Seperating the input data and labels
train_data = (train[:,0:784])/255
train_labels = train[:,784]

test_data = (test[:,0:784])/255
test_labels = test[:,784]

from keras.utils.np_utils import to_categorical

train_labels = to_categorical(train_labels)
test_labels = to_categorical(test_labels)

In [3]:
print (train_data.shape)
print (train_labels.shape)
print (test_data.shape)
print (test_labels.shape)

(60000, 784)
(60000, 10)
(10000, 784)
(10000, 10)


In [4]:
# Training an autoencoder model on cifar-10 PCA reduced data

input_img = Input(shape=(784,))
crrpt_img = Dropout(0.5)(input_img)
encoded = Dense(1000, activation='sigmoid')(crrpt_img)
decoded = Dense(784, activation='linear')(encoded)

autoencoder = Model(input_img,decoded)

nb_epoch = 10
batch_size = 32

autoencoder.compile(optimizer='adam',
                    loss='mean_squared_error',
                    metrics=['mse'])


In [5]:
history = autoencoder.fit(train_data, train_data,
                    epochs=nb_epoch,
                    batch_size=batch_size,
                    shuffle=True,
                    validation_data=(test_data, test_data),
                    verbose=1)


autoencoder.save('DAE_l1_model.h5')


Train on 60000 samples, validate on 10000 samples
Epoch 1/10
60000/60000 [==============================] - 92s - loss: 0.0247 - mean_squared_error: 0.0247 - val_loss: 0.0128 - val_mean_squared_error: 0.0128
Epoch 2/10
60000/60000 [==============================] - 88s - loss: 0.0186 - mean_squared_error: 0.0186 - val_loss: 0.0110 - val_mean_squared_error: 0.0110
Epoch 3/10
60000/60000 [==============================] - 84s - loss: 0.0161 - mean_squared_error: 0.0161 - val_loss: 0.0100 - val_mean_squared_error: 0.0100
Epoch 4/10
60000/60000 [==============================] - 90s - loss: 0.0145 - mean_squared_error: 0.0145 - val_loss: 0.0093 - val_mean_squared_error: 0.0093
Epoch 5/10
60000/60000 [==============================] - 91s - loss: 0.0136 - mean_squared_error: 0.0136 - val_loss: 0.0088 - val_mean_squared_error: 0.0088
Epoch 6/10
60000/60000 [==============================] - 91s - loss: 0.0130 - mean_squared_error: 0.0130 - val_loss: 0.0085 - val_mean_squared_error: 0.0085
Ep

In [ ]:
#from keras.layers import load_model
#autoencoder1 = load_model('DAE_l1_model.h5')

In [6]:
encoder = Model(input_img,encoded)
htrain_data = encoder.predict(train_data)


In [7]:
input_img1 = Input(shape=(1000,))
crrpt_img1 = Dropout(0.5)(input_img1)
encoded1 = Dense(1000, activation='sigmoid')(crrpt_img1)
decoded1 = Dense(1000, activation='sigmoid')(encoded1)

autoencoder1 = Model(input_img1,decoded1)

autoencoder1.compile(optimizer='adam',
                    loss='binary_crossentropy',
                    metrics=['binary_crossentropy'])

autoencoder1.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 1000)              0         
_________________________________________________________________
dropout_2 (Dropout)          (None, 1000)              0         
_________________________________________________________________
dense_3 (Dense)              (None, 1000)              1001000   
_________________________________________________________________
dense_4 (Dense)              (None, 1000)              1001000   
Total params: 2,002,000
Trainable params: 2,002,000
Non-trainable params: 0
_________________________________________________________________


In [8]:
history = autoencoder1.fit(htrain_data, htrain_data,
                    epochs=nb_epoch,
                    batch_size=batch_size,
                    shuffle=True,
                    verbose=1)


autoencoder1.save('DAE_l2_model.h5')


Epoch 1/10
60000/60000 [==============================] - 145s - loss: 0.1570 - binary_crossentropy: 0.1570     ETA: 16 - ETA: 14s - loss: 0
Epoch 2/10
60000/60000 [==============================] - 138s - loss: 0.1470 - binary_crossentropy: 0.1470   
Epoch 3/10
60000/60000 [==============================] - 149s - loss: 0.1447 - binary_crossentropy: 0.1447   
Epoch 4/10
60000/60000 [==============================] - 138s - loss: 0.1433 - binary_crossentropy: 0.1433   
Epoch 5/10
60000/60000 [==============================] - 122s - loss: 0.1426 - binary_crossentropy: 0.1426   
Epoch 6/10
60000/60000 [==============================] - 107s - loss: 0.1421 - binary_crossentropy: 0.1421   
Epoch 7/10
60000/60000 [==============================] - 107s - loss: 0.1418 - binary_crossentropy: 0.1418   
Epoch 8/10
60000/60000 [==============================] - 111s - loss: 0.1416 - binary_crossentropy: 0.1416   
Epoch 9/10
60000/60000 [==============================] - 123s - loss: 0.1414 - bi

In [ ]:
#from keras.layers import load_model
#autoencoder1 = load_model('DAE_l2_model.h5')

In [9]:
# Training with unsupervised initialization for layer-1 
# of an MLP using autoencoder weights
from keras.layers import BatchNormalization

mlp = Sequential()
mlp.add(Dropout(0.2, input_shape=(784,)))
mlp.add(Dense(1000, activation='sigmoid'))
mlp.add(Dropout(0.5))
mlp.add(Dense(1000, activation='sigmoid'))
mlp.add(Dropout(0.5))
mlp.add(Dense(10, activation='softmax'))



mlp.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

mlp.layers[1].set_weights(autoencoder.layers[2].get_weights())
mlp.layers[3].set_weights(autoencoder1.layers[2].get_weights())


history = mlp.fit(train_data[:25000], train_labels[:25000],
                    batch_size=batch_size,
                    epochs=nb_epoch,
                    verbose=1,
                    validation_data=(test_data, test_labels))

Train on 25000 samples, validate on 10000 samples
Epoch 1/10
25000/25000 [==============================] - 47s - loss: 0.3992 - acc: 0.8828 - val_loss: 0.1416 - val_acc: 0.9561
Epoch 2/10
25000/25000 [==============================] - 47s - loss: 0.1814 - acc: 0.9450 - val_loss: 0.1044 - val_acc: 0.9665
Epoch 3/10
25000/25000 [==============================] - 47s - loss: 0.1420 - acc: 0.9562 - val_loss: 0.0857 - val_acc: 0.9720
Epoch 4/10
25000/25000 [==============================] - 47s - loss: 0.1207 - acc: 0.9624 - val_loss: 0.0798 - val_acc: 0.9763
Epoch 5/10
25000/25000 [==============================] - 46s - loss: 0.1060 - acc: 0.9663 - val_loss: 0.0754 - val_acc: 0.9759
Epoch 6/10
25000/25000 [==============================] - 47s - loss: 0.0905 - acc: 0.9721 - val_loss: 0.0736 - val_acc: 0.9778
Epoch 7/10
25000/25000 [==============================] - 47s - loss: 0.0824 - acc: 0.9735 - val_loss: 0.0734 - val_acc: 0.9770
Epoch 8/10
25000/25000 [==============================

In [10]:
test_data.shape

(10000, 784)

In [11]:
len(test_labels)

10000